In [1]:
import os
import sys
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cifar10_input
from pgd_attack import PGDAttackClassifier, PGDAttackDetector
from model import Model, BayesClassifier
from eval_utils import BaseDetectorFactory, batched_run, eps8_attack_config

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
tf.logging.set_verbosity(tf.logging.ERROR)

np.random.seed(123)


In [2]:
robust_classifier = Model(mode='eval', var_scope='classifier')
classifier_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                    scope='classifier')
classifier_saver = tf.train.Saver(var_list=classifier_vars)
classifier_checkpoint = 'models/adv_trained_prefixed_classifier/checkpoint-70000'

factory = BaseDetectorFactory()

cifar = cifar10_input.CIFAR10Data('cifar10_data')

num_eval_examples = 10000
eval_data = cifar.eval_data
x_test = eval_data.xs.astype(np.float32)[:num_eval_examples]
y_test = eval_data.ys.astype(np.int32)[:num_eval_examples]

In [3]:
sess = tf.Session()

In [4]:
classifier_saver.restore(sess, classifier_checkpoint)
factory.restore_base_detectors(sess)

base_detectors = factory.get_base_detectors()
bayes_classifier = BayesClassifier(base_detectors)

In [5]:
grad1 = tf.gradients(robust_classifier.xent, robust_classifier.x_input)[0]
%timeit sess.run(grad1, feed_dict={robust_classifier.x_input: x_test[:128], \
                                   robust_classifier.y_input: y_test[:128]})

683 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:

grad2 = tf.gradients(bayes_classifier.xent, bayes_classifier.x_input)[0]
%timeit sess.run(grad2, feed_dict={bayes_classifier.x_input: x_test[:30], \
                                   bayes_classifier.y_input: y_test[:30]})

1.85 s ± 42.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit sess.run(robust_classifier.predictions, feed_dict={robust_classifier.x_input: x_test[:1000], \
                                                           robust_classifier.y_input: y_test[:1000]})

2.09 s ± 35.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit sess.run(bayes_classifier.predictions, feed_dict={bayes_classifier.x_input: x_test[:1000], \
                                                           bayes_classifier.y_input: y_test[:1000]})

19.7 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
sess.run(robust_classifier.accuracy, feed_dict={robust_classifier.x_input: x_test[:1000], \
                                                           robust_classifier.y_input: y_test[:1000]})

0.873

In [10]:
sess.run(bayes_classifier.accuracy, feed_dict={bayes_classifier.x_input: x_test[:1000], \
                                                           bayes_classifier.y_input: y_test[:1000]})

0.877